This is script for beta calculation. 

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np

def calculate_beta(stock_returns, market_returns):
    covariance_matrix = np.cov(stock_returns, market_returns)
    beta = covariance_matrix[0, 1] / covariance_matrix[1, 1]
    return beta

def get_stock_data(ticker, start_date):
    stock_data = yf.download(ticker, start=start_date, progress=False)
    stock_data['Returns'] = stock_data['Adj Close'].pct_change()
    return stock_data

def calculate_betas(stock_tickers, reference_ticker, lookback_periods):
    end_date = pd.to_datetime('today')
    results = {}
    
    for stock in stock_tickers:
        results[stock] = {}
        for days in lookback_periods:
            start_date = (end_date - pd.Timedelta(days=days)).strftime('%Y-%m-%d')
            
            stock_data = get_stock_data(stock, start_date)
            reference_data = get_stock_data(reference_ticker, start_date)
            
            merged_data = pd.merge(stock_data['Returns'], reference_data['Returns'], left_index=True, right_index=True, suffixes=('_stock', '_reference')).dropna()
            
            stock_returns = merged_data['Returns_stock']
            reference_returns = merged_data['Returns_reference']
            
            beta = calculate_beta(stock_returns, reference_returns)
            results[stock][f'{days}_days'] = beta
    
    return pd.DataFrame(results)

# Define the stocks and the reference stock
stock_tickers = ['BITO', 'MARA', 'RIOT']
reference_ticker = 'BTC-USD'
lookback_periods = [30, 60, 90, 180, 365]

# Calculate the betas
betas = calculate_betas(stock_tickers, reference_ticker, lookback_periods)

# Transpose the DataFrame for better readability
betas_transposed = betas.transpose()

# Display the results
print("Calculating betas for --> ", reference_ticker)
print(betas_transposed)


Calculating betas for -->  BTC-USD
       30_days   60_days   90_days  180_days  365_days
BITO  0.879911  1.020816  1.056173  0.948342  0.957484
MARA  1.025776  1.130912  1.087311  0.950420  1.167425
RIOT  0.919784  1.077919  1.012647  0.948330  0.981441
